<a href="https://colab.research.google.com/github/singer-yang/AutoLens/blob/main/auto_lens_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone 'https://github.com/singer-yang/AutoLens.git'
!pip install transformers
import sys, os
sys.path.insert(0, os.path.abspath('./AutoLens'))

fatal: destination path 'AutoLens' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00


In [4]:
""" Aotumated lens design with curriculum learning, using RMS errors as loss function.
"""
import os
import string
import argparse
import logging
import yaml
from datetime import datetime
import torch
import deeplens
from deeplens.utils import *
from deeplens.optics import create_lens

def config():
    """ Config file for training.
    """
    # Config file
    with open('./AutoLens/configs/auto_lens_design.yml') as f:
        args = yaml.load(f, Loader=yaml.FullLoader)

    # Result dir
    characters = string.ascii_letters + string.digits
    random_string = ''.join(random.choice(characters) for i in range(4))
    current_time = datetime.now().strftime("%m%d-%H%M%S")
    exp_name = current_time + '-CurriculumLensDesign-' + random_string
    result_dir = f'./results/{exp_name}'
    os.makedirs(result_dir, exist_ok=True)
    args['result_dir'] = result_dir

    set_seed(args['seed'])

    # Log
    set_logger(result_dir)
    logging.info(f'EXP: {args["EXP_NAME"]}')

    # Device
    num_gpus = torch.cuda.device_count()
    args['num_gpus'] = num_gpus
    device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
    args['device'] = device
    logging.info(f'Using {num_gpus} GPUs')

    return args


def change_lens(lens, diag, fnum):
    """ Change lens for each curriculum step.
    """
    # sensor
    lens.r_last = diag / 2
    lens.hfov = np.arctan(lens.r_last / lens.foclen)

    # aperture
    lens.fnum = fnum
    aper_r = lens.foclen / fnum / 2
    lens.surfaces[lens.aper_idx].r = aper_r

    return lens


def curriculum_learning(lens, args):
    """ Curriculum learning for lens design.
    """
    lrs = [float(lr) for lr in args['lrs']]

    curriculum_steps = args['curriculum_steps']
    fnum_target = args['FNUM'] * 0.95
    fnum_start = args['FNUM_START']
    diag_target = args['DIAG'] * 1.05
    diag_start = args['DIAG_START']

    for step in range(args['curriculum_steps']+1):

        # ==> Design target for this step
        args['step'] = step
        diag1 = diag_start + (diag_target - diag_start) * np.sin(step / curriculum_steps * np.pi/2)
        fnum1 = fnum_start + (fnum_target - fnum_start) * np.sin(step / curriculum_steps * np.pi/2)
        lens = change_lens(lens, diag1, fnum1)

        lens.analysis(save_name=f'{result_dir}/step{step}_starting_point', zmx_format=True)
        lens.write_lensfile(f'{result_dir}/step{step}_starting_point.txt', write_zmx=True)
        logging.info(f'==> Curriculum learning step {step}, target: FOV {round(lens.hfov * 2 * 57.3, 2)}, DIAG {round(2 * lens.r_last, 2)}mm, F/{lens.fnum}.')

        # ==> Lens design using RMS errors
        iterations = 1000
        lens.refine(lrs=lrs, decay=args['ai_lr_decay'], iterations=iterations, test_per_iter=50, importance_sampling=False, result_dir=result_dir)

    # ==> Refine lens at the last step
    lens.refine(iterations=5000, test_per_iter=100, centroid=True, importance_sampling=True, result_dir=result_dir)
    logging.info('==> Training finish.')

    # ==> Final lens
    lens = change_lens(lens, args['DIAG'], args['FNUM'])




In [ ]:
args = config()
result_dir = args['result_dir']
device = args['device']

# =====> 1. Create a lens
create_lens(rff=float(args['rff']), flange=float(args['flange']), d_aper=args['d_aper'], hfov=args['HFOV'], imgh=args['DIAG'], fnum=args['FNUM'], surfnum=args['element'], dir=result_dir)
lens_name = f'./{result_dir}/starting_point_hfov{args["HFOV"]}_imgh{args["DIAG"]}_fnum{args["FNUM"]}.txt'
lens = deeplens.Lensgroup(filename=lens_name)
for i in lens.find_diff_surf():
    lens.surfaces[i].init_c()
    lens.surfaces[i].init_ai(args['ai_degree'])
    lens.surfaces[i].init_k()
    lens.surfaces[i].init_d()

# =====> 2. Set lens design targets
lens.set_target_fov_fnum(hfov=args['HFOV'], fnum=args['FNUM'], imgh=args['DIAG'])
logging.info(f'==> Design target: FOV {round(args["HFOV"]*2*57.3, 2)}, DIAG {args["DIAG"]}mm, F/{args["FNUM"]}, FOCLEN {round(args["DIAG"]/2/np.tan(args["HFOV"]), 2)}mm.')
lens.analysis(save_name=f'{result_dir}/lens_starting_point')

# =====> 3. Lens design with RMS errors
curriculum_learning(lens, args)

# =====> 4. Analyze final result
lens.prune()
lens.post_computation()

logging.info(f'Actual: FOV {lens.hfov}, IMGH {lens.r_last}, F/{lens.fnum}.')
lens.write_lensfile(f'{result_dir}/final_lens.txt', write_zmx=True)
lens.analysis(save_name=f'{result_dir}/final_lens', zmx_format=True)

INFO:root:EXP: 6P, 80deg, F/2.0, 7.66mm sensor, lens design
2023-09-23 14:35:25:INFO:EXP: 6P, 80deg, F/2.0, 7.66mm sensor, lens design
INFO:root:Using 1 GPUs
2023-09-23 14:35:25:INFO:Using 1 GPUs
INFO:root:==> Design target: FOV 80.22, DIAG 7.66mm, F/2.0, FOCLEN 4.55mm.
2023-09-23 14:35:34:INFO:==> Design target: FOV 80.22, DIAG 7.66mm, F/2.0, FOCLEN 4.55mm.
DEBUG:matplotlib.font_manager:findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/STIXNonUni.ttf', name='STIXNonUnicode', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/STIXSizTwoSymBol.ttf', name='STIXSizeTwoSym', style='normal', variant='normal', weight=700, stretch='normal', size

Avg RMS spot size (radius): 805.273um, On-axis RMS radius: 805.903um, Off-axis RMS radius: 801.55um


INFO:root:==> Curriculum learning step 0, target: FOV 47.49, DIAG 4.0mm, F/2.8.
2023-09-23 14:35:42:INFO:==> Curriculum learning step 0, target: FOV 47.49, DIAG 4.0mm, F/2.8.
INFO:root:lr:[0.0005, 0.0001, 0.1, 0.0001], decay:0.1, iterations:1000, spp:256, M:31.
2023-09-23 14:35:42:INFO:lr:[0.0005, 0.0001, 0.1, 0.0001], decay:0.1, iterations:1000, spp:256, M:31.


Avg RMS spot size (radius): 575.068um, On-axis RMS radius: 575.548um, Off-axis RMS radius: 575.921um


Progress:   0%|          | 0/1001 [00:00<?, ?it/s, rms=0]

...

Progress:   5%|▍         | 50/1001 [01:46<29:48,  1.88s/it, rms=0.0131]

Avg RMS spot size (radius): 205.373um, On-axis RMS radius: 257.676um, Off-axis RMS radius: nanum


Progress:  10%|▉         | 100/1001 [03:30<30:29,  2.03s/it, rms=0.00106]

Avg RMS spot size (radius): 36.837um, On-axis RMS radius: 64.23um, Off-axis RMS radius: nanum


Progress:  10%|█         | 104/1001 [03:44<39:36,  2.65s/it, rms=0.000887]